In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.manifold import TSNE

In [3]:
DATASET_DIR = "../dataset/regression/concrete"
SAVE_NAME = "concrete_ori.csv"

In [4]:
df = pd.read_csv(f"{DATASET_DIR}/{SAVE_NAME}")
df

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Concrete compressive strength
0,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
1,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
3,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075
4,266.0,114.0,0.0,228.0,0.0,932.0,670.0,90,47.029847
...,...,...,...,...,...,...,...,...,...
1024,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.284354
1025,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.178794
1026,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.696601
1027,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.768036


In [5]:
y_feature = "Concrete compressive strength"

In [6]:
df.corr().iloc[-1,:].sort_values()

Water                           -0.288356
Coarse Aggregate                -0.167779
Fine Aggregate                  -0.164798
Fly Ash                         -0.103966
Blast Furnace Slag               0.137545
Age                              0.330735
Superplasticizer                 0.369022
Cement                           0.494620
Concrete compressive strength    1.000000
Name: Concrete compressive strength, dtype: float64

In [7]:
df.corr()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Concrete compressive strength
Cement,1.000000,-0.274046,-0.396759,-0.079606,0.094565,-0.111809,-0.220606,0.082872,0.494620
Blast Furnace Slag,-0.274046,1.000000,-0.324504,0.106604,0.042883,-0.283483,-0.282912,-0.044496,0.137545
Fly Ash,-0.396759,-0.324504,1.000000,-0.257994,0.377031,-0.009273,0.078157,-0.154660,-0.103966
Water,-0.079606,0.106604,-0.257994,1.000000,-0.658409,-0.181683,-0.452230,0.277479,-0.288356
Superplasticizer,0.094565,0.042883,0.377031,-0.658409,1.000000,-0.265928,0.221969,-0.192929,0.369022
Coarse Aggregate,-0.111809,-0.283483,-0.009273,-0.181683,-0.265928,1.000000,-0.177677,-0.002782,-0.167779
Fine Aggregate,-0.220606,-0.282912,0.078157,-0.452230,0.221969,-0.177677,1.000000,-0.156544,-0.164798
Age,0.082872,-0.044496,-0.154660,0.277479,-0.192929,-0.002782,-0.156544,1.000000,0.330735
Concrete compressive strength,0.494620,0.137545,-0.103966,-0.288356,0.369022,-0.167779,-0.164798,0.330735,1.000000


In [8]:
df["Superplasticizer"].value_counts()

0.0     379
8.0      27
11.6     21
7.0      19
6.0      17
       ... 
6.6       1
12.2      1
15.6      1
20.8      1
2.5       1
Name: Superplasticizer, Length: 155, dtype: int64

In [15]:
df["Age"].value_counts()

28     424
3      134
7      126
56      91
14      62
90      54
100     52
180     26
91      22
365     14
270     13
360      6
120      3
1        2
Name: Age, dtype: int64

In [33]:
df["Water"].value_counts()

192.0    118
228.0     54
185.7     46
203.5     36
186.0     28
        ... 
218.0      1
177.0      1
127.0      1
198.7      1
184.4      1
Name: Water, Length: 205, dtype: int64

In [17]:
df["Cement"].value_counts()

425.00    20
362.60    20
251.37    15
446.00    14
310.00    14
          ..
259.90     1
139.90     1
136.40     1
266.20     1
321.30     1
Name: Cement, Length: 280, dtype: int64

In [19]:
pd.qcut(df["Cement"], 3, labels=['0', '1', '2']).value_counts()

0    344
2    343
1    342
Name: Cement, dtype: int64

In [34]:
pd.qcut(df["Water"], 3, labels=['0', '1', '2']).value_counts()

1    427
0    349
2    253
Name: Water, dtype: int64

In [29]:
df_labels12 = df[df["Superplasticizer"]>0]
pd.qcut(df_labels12["Superplasticizer"], 2, labels=['1', '2']).value_counts()

1    326
2    324
Name: Superplasticizer, dtype: int64

In [32]:
domain = pd.Series(0,index=df.index)
domain[df_labels12.index] = pd.qcut(df_labels12["Superplasticizer"], 2, labels=['1', '2'])
domain.value_counts()

0    379
1    326
2    324
dtype: int64